
# True positive proportions in fMRI clusters using Notip

This script showcases the so-called Notip procedure [1], in
which the proportion of true discoveries in arbitrary clusters is estimated.
The clusters can be defined from the input image, i.e. in a circular way, as
the error control accounts for arbitrary cluster selection.


In [ ]:
pip install notip -q

## Fetch dataset

We download a list of image of the HCP dataset showcasing activity during a working memory task [0]. Concretely, these maps represent brain activation when subjects perform a 2-back task with face images. Note that we fetch individual t-maps that represent the BOLD activity estimate divided by the uncertainty about this estimate.

In [ ]:
# fetch the data from neurovault
from nilearn.datasets import fetch_neurovault

image_terms={'contrast_definition':'2BK_FACE'} # which contrast we want
n_subjects = 30 # number of desired images
nv_data = fetch_neurovault(
    collection_id=4337,     # this collection
    image_terms=image_terms,# image selection
    max_images=n_subjects,
    # resample=True,          # resample to 3mm for the sake of time
)
imgs = nv_data["images"]

Let's visualize the input data:

In [ ]:
import matplotlib.pyplot as plt
from nilearn.plotting import plot_glass_brain

fig, axes = plt.subplots(nrows=6, ncols=5, figsize=(6, 6), dpi=300)
# we use a glass brain to see each map in one glance
for cidx, tmap in enumerate(imgs):
    plot_glass_brain(
        tmap,
        colorbar=False,
        threshold=2.0,
        axes=axes[int(cidx / 5), int(cidx % 5)],
        plot_abs=False,
        annotate=False,
        display_mode='z')

To extract signal from these images, we need a masker. While loading the data, we smooth them with an 8mm-kernel to improve sensitivity. 

In [ ]:
from nilearn.maskers import NiftiMasker

smoothing_fwhm = 8.0
nifti_masker = NiftiMasker(smoothing_fwhm=smoothing_fwhm)

In [ ]:
fmri_input = nifti_masker.fit_transform(imgs)

In [ ]:
print(fmri_input.shape)

In [ ]:
# Note that the masker has automatically computed a brain mask
# One can take a look at it
from nilearn.plotting import plot_roi
plot_roi(nifti_masker.mask_img_)

We have extracted the values of 255612 voxels from 30 images.

### Computing True Discovery Proportion (TDP) lower bounds on data-derived clusters

First, we need to compute a statistical map from the input data. This is done via a t-test, performed for each voxel.

In [ ]:
from scipy.stats import norm, ttest_1samp

# Let's run a one-sample t test on these data
stats_, p_values = ttest_1samp(
    fmri_input, 0, alternative='greater')

# Let's make this an image by using the inverse_transform method of the masker
stat_map = nifti_masker.inverse_transform(stats_)

p_values_map = nifti_masker.inverse_transform(p_values)

Let us visualise the resulting t map:

In [ ]:
from nilearn.plotting import view_img
stat_threshold = 3.5
view_img(stat_map, 
         title='Group-level t-map',
         threshold=stat_threshold,
         colorbar=True)

This map makes sense.
We can now use Notip to compute TDP lower bounds on clusters exceeding a z-value threshold. We set this threshold to 3.5.

In [ ]:
from notip.posthoc_fmri import get_clusters_table_TDP_1samp
get_clusters_table_TDP_1samp(
    fmri_input,
    n_permutations=200,
    stat_threshold=stat_threshold,
    methods=['Notip'],
    nifti_masker=nifti_masker)

We have reduced the number of permutations to 200 for the sake of computation time. Note that we can get tighter FDP control by increasing this number.

### Comparison with other TDP lower bounds 

There exist other approach to get TDP estimates. Among those, All-Resolution-Inference (ARI) does not require permutations. Let us compare their result.

In [ ]:
from notip.posthoc_fmri import get_clusters_table_TDP_1samp
get_clusters_table_TDP_1samp(
    fmri_input, 
    n_permutations=200,
    stat_threshold=stat_threshold,
    methods=['ARI', 'Notip'],
    nifti_masker=nifti_masker)

### Using Notip on regions from atlases

In [ ]:
from nilearn import datasets
# atlas = datasets.fetch_atlas_harvard_oxford('cort-prob-2mm')

atlas = datasets.fetch_atlas_harvard_oxford("cort-maxprob-thr0-2mm")
atlas_masker = NiftiMasker(smoothing_fwhm=None)

atlas_filename = atlas.maps
labels = atlas.labels
atlas_masked = atlas_masker.fit_transform(atlas_filename)

# project the data onto the same space

fmri_input_atlas = atlas_masker.transform(imgs)

We have 48 atlas regions.

In [ ]:
labels

In [ ]:
import numpy as np
from nilearn.image import math_img

# Find the index of the Superior Parietal Lobule
idx = np.where(np.array(labels) == 'Superior Parietal Lobule')[0]

# make it a binary region definition
region_mask =  math_img(f'i1 == {idx}', i1=atlas_filename)
plot_roi(region_mask)

In [ ]:
from notip.posthoc_fmri import tdp_bound_notip_1samp
notip_bound, cluster_map = tdp_bound_notip_1samp(
    fmri_input_atlas,
    region_mask,
    n_permutations=200,
    nifti_masker=atlas_masker)

In [ ]:
from nilearn.plotting import plot_stat_map
plot_stat_map(cluster_map, title='TDP > {0:.2f}'.format(notip_bound))

### Using Notip on user-defined clusters

We will now use Notip on clusters extracted from the data; we seek to find connected components exceeding a z-value threshold.

In [ ]:
from nilearn.regions import connected_regions
from nilearn.image import binarize_img, index_img

# binarize the thresholded stat map
th_img = binarize_img(stat_map, 3.5)

# Extract the connected components
label_map, indices = connected_regions(
    th_img,
    min_region_size=1350,
    extract_type='connected_components',
    mask_img=None)

# Get the spatial map corresponding to the first cluster
# i.e. with index = 0
cluster_mask = index_img(label_map, 0)

# look at it
plot_roi(cluster_mask)

In [ ]:
np.count_nonzero(cluster_mask.get_fdata())

This cluster comprises 1453 voxels.
Now, let's infer the TDP on this region. This looks circular ---it is actually--- as the region was defined from the map in the first place, but in the present case, this is legal, as we use a post-hoc inference scheme.

In [ ]:
from notip.posthoc_fmri import tdp_bound_notip_1samp
notip_bound, cluster_map = tdp_bound_notip_1samp(
    fmri_input,
    cluster_mask,
    n_permutations=200,
    nifti_masker=nifti_masker)

Let's visualize the results:

In [ ]:
plot_stat_map(
    cluster_map, title='TDP > {0:.2f}'.format(notip_bound))

## FDP control vs FDR control

We can also apply Notip to obtain a TDP lower bound on the 5% Benjamini-Hochberg region:

In [ ]:
from nilearn.glm import fdr_threshold
from scipy.stats import norm
from nilearn.image import threshold_img, math_img
q = 0.05

# fdr_threshold takes z-values as input,
# so we need to convert p-values to z-values
z_values = norm.isf(p_values)
bh_region = z_values >= fdr_threshold(z_values, q)

stat_threshold_bh = np.min(stats_[bh_region])
print('BH threshold: {0:.2f}'.format(stat_threshold_bh))

plot_stat_map(
    stat_map, 
    threshold=stat_threshold_bh, 
    title='TDR > {0:.2f}'.format(1 - q))

In [ ]:
# To infer on the TDP in this region, we binarize the stat map
bh_mask = binarize_img(stat_map, stat_threshold_bh)

# Then we call the TDP inference function on this particular region
notip_bound, cluster_map = tdp_bound_notip_1samp(
    fmri_input,
    bh_mask,
    n_permutations=200,
    nifti_masker=nifti_masker)

In [ ]:
plot_stat_map(cluster_map, title='TDP > {0:.2f}'.format(notip_bound))

Note that Notip only guarantees that, with high probability, about half of voxels at least are active in this region: FDR control does not imply FDP control at the same level.

# References

Blain, Alexandre, Bertrand Thirion, and Pierre Neuvial. "Notip: Non-parametric True Discovery Proportion control for brain imaging." NeuroImage 260 (2022): 119492. doi:https://doi.org/10.1016/j.neuroimage.2022.119492

Dimitri Papadopoulos Orfanos, Vincent Michel, Yannick Schwartz, Philippe Pinel, Antonio Moreno, Denis Le Bihan, and Vincent Frouin. The brainomics/localizer database. NeuroImage, 144:309–314, 2017. Data Sharing Part II. URL: https://www.sciencedirect.com/science/article/pii/S1053811915008745, doi:https://doi.org/10.1016/j.neuroimage.2015.09.052.